In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import matplotlib.pylab as pylab
import ipywidgets as widgets
from os import path
from sklearn.decomposition import PCA
from matplotlib.offsetbox import AnchoredText
from IPython.display import clear_output
from IPython.display import display
%matplotlib inline

In [74]:
class DataLoader:
    def __init__(self):
        self.__kall = None
        self.__rsem = {}
        
    def __loadKall(self):
        kallDF = pd.read_csv('/home/lima/Project/simulation/Comparision/Kall/Real/KallistoReadCounts.csv')
        kallDF = kallDF.rename(columns = {"Unnamed: 0": "GeneID"})
        kallDF.set_index("GeneID", inplace = True)
        self.__kall = kallDF
    
    def __loadRSEM(self):
        cutOff = [0, 2.5, 5, 7.5, 10]
        # key of the tempDict is the cut off threshould, value is the cnt matrix accordingly
        tempDict = {}
        dictList = []
        for cut in cutOff:            
            fullPath = "/home/lima/Project/simulation/Comparision/RSEM/Real/CountsCutOffValue{cut}.csv".format(cut = cut)
            dataFrame = pd.read_csv(fullPath)
            dataFrame = dataFrame.rename(columns = {"Unnamed: 0": "GeneID"})
            dataFrame.set_index("GeneID", inplace = True)
            
            tempDict[cut] = dataFrame
        self.__rsem = tempDict
    
    def getKall(self):
        self.__loadKall()
        return self.__kall
    
    def getRSEM(self):
        self.__loadRSEM()
        return self.__rsem

In [88]:
class ConcateData:
    def __init__(self, groupList, rsem, kallisto):
        # input:
        # @groupList: a list contains the names of samples. there are four groups of samples, high, low, veh, pt
        # @rsem: rsem counts
        # @kallisto: kallisto counts
        # both counts are in gene level
        self.__list = groupList
        self.__rsem = rsem
        self.__kallisto = kallisto
        self.__result = []
        
    def __concat(self):        
        
        for sample in self.__list:
            targetRSEM = self.__rsem[[sample]]
            targetKall = self.__kallisto[[sample]]
            # print(targetRSEM)
            # print(targetKall)
            rsemLen = len(targetRSEM)
            kallLen = len(targetKall)
        
            print("{num} genes have been detected in {sample} by RSEM".format(num = rsemLen, sample = sample))
            print("{num} genes have been detected in {sample} by kallisto".format(num = kallLen, sample = sample))
            
            # convert the index, gene ids, to the first column for merge two dataframes
            targetRSEM.reset_index(inplace = True)
            targetKall.reset_index(inplace = True)
            # print(targetKall)
            # print(targetRSEM)
            df = pd.merge(targetRSEM, targetKall, how = 'outer', on = 'GeneID', suffixes = ('_RSEM', '_Kallisto'))
            print("{num} genes in both dataframes".format(num = len(df)))
            # take out the GeneIDs in each data frame and put them into two sets
            rsemIdx = set(targetRSEM['GeneID'])
            kallIdx = set(targetKall['GeneID'])
            
            inBoth = rsemIdx & kallIdx
            rsemOnly = rsemIdx - inBoth
            kallOnly = kallIdx - inBoth
            
            inBoth = list(inBoth)
            rsemOnly = list(rsemOnly)
            kallOnly = list(kallOnly)
            
            bothDF = pd.DataFrame(list(zip(inBoth, ["DetectedByBoth"] * len(inBoth))), columns = ["GeneID", "Label"])
            rsemOnlyDF = pd.DataFrame(list(zip(rsemOnly, ["OnlyDetectedByRSEM"] * len(rsemOnly))), columns = ["GeneID", "Label"])
            kallOnlyDF = pd.DataFrame(list(zip(kallOnly, ["OnlyDetectedByKallisto"] * len(kallOnly))), columns = ["GeneID", "Label"])
            labelDF = pd.concat([bothDF, rsemOnlyDF, kallOnlyDF])
            print("{num} labels have been attached".format(num = len(labelDF)))
            # fill nans in df with 0
            df.fillna(0, inplace = True)
            df = pd.merge(df, labelDF, how = 'inner', on = 'GeneID')
            #print(df)
            self.__result.append(df)
            
    def setData(self):
        self.__concat()
        
    def getData(self):
        return self.__result

In [ ]:
class Plotter:
    def __init__(self, pairedData):
        self.__data = pairedData
        
    def plotPlotBlock(self, df):
        
        params = {'legend.fontsize': 8, 
                    'font.family': 'serif',
                    'font.weight': 'medium',
                    'font.variant': 'normal',
                    'figure.figsize': (10, 10), 
                    'figure.edgecolor': '#04253a', 
                    'figure.titlesize': 8, 
                    'axes.labelsize': 6, 
                    'axes.titlesize': 6, 
                    'xtick.labelsize': 6, 
                    'ytick.labelsize': 6}
        pylab.rcParams.update(params)
        figureOne, axes = plt.subplots(2, 2, constrained_layout = True)
        # self.buildPlotBlock(df, axes, cutOff, names)
        self.buildPlotBlock(df, axes)
        # plt.suptitle('Statistical plots for comparing {realData} & {simulatedData} with reads shorter than {cut} being removed'.format(realData = names[0], simulatedData = names[1], cut = cutOff))
        # title = 'Real data and synthetic data of {realData} with reads shorter than {cut} being removed.png'.format(realData = names[0], cut = cutOff)
        plt.savefig(title, dpi = 300, bbox_inches = 'tight')
        plt.plot()
        
    def buildPlotBlock(self, df, axes):
        """
        buildPlotBlock is a helper function helping putting the desired plots, such as the scatter plot, box plot,
        violin plot and the distribution plot into a grided pyplot
    
        Input:
        @df: the dataframe which provides the data
        @axes: a 2x2 subplot matrix 
        """
        # df = np.log2(df[names] + 1)
        sns.set(style = "whitegrid", palette = "muted", color_codes = True)
        # put the violin plot to axes[0, 0]
        axes[0, 0].clear()
        sns.violinplot(data = df, linewidth = 0.2, palette = "husl", ax = axes[0, 0])
    
        # put the boxplot to axes[1, 1]
        axes[1, 1].clear()
        sns.boxplot(data = df, linewidth = 0.2, palette = "husl", ax = axes[1, 1])
    
        # put the distribution plot to axes[0, 1]
        axes[0, 1].clear()
        sns.distplot(df.iloc[ : , 0], kde = True, color = "b", label = 'Real', ax = axes[0, 1])
        sns.distplot(df.iloc[ : , 1], kde = True, color = "r", label = 'Simulated', ax = axes[0, 1])
        axes[0, 1].legend(loc = 'best', prop = {'size': 8})
        axes[0, 1].set_xlabel("Distribution plot for real data and simulated data")
    
        # put the scatter plot to axes [1, 0]
        axes[1, 0].clear()
        self.scatterPlot(df, axes[1, 0], cutOff, names)
                         
    def scatterPlot(self, df, ax):
        """
        the scatterPlot function plots a pairwise scatter plot for the simulated data and the real data for the same 
        sample, with the x-coordinates being the counts number of the real data and the y-coordinates being the counts
        number of the simulated data
    
        input data:
        @df: a pandas dataframe contains the counts for the real and the simulated data
        @ax: an axes object
        @names: a list contains the sample names
        """
    
        # notInrealData, notInSimulatedData, inNeither are lists stores the row indecies for data as theirs names
        # notInRealData = df.index[df.iloc[ : , 0] == 0].tolist()
        # notInSimulatedData = df.index[df.iloc[ : , 1] == 0].tolist()
        # inNeither = list(set(notInRealData) & set(notInSimulatedData))
    
        # inBoth, simOnly, realOnly, neither are integers for 
        # genes found in both samples, one of the samples or none of them
        # neither = len(inNeither)
        # simOnly = len(notInRealData) - neither
        # realOnly = len(notInSimulatedData) - neither
        # inBoth = len(df) - simOnly - realOnly + neither
    
        # category the genes into 4 classes
        # df["ExpressionStatus"] = ["BothSamples"] * len(df)
        # df.loc[notInRealData, "ExpressionStatus"] = "SimulatedOnly"
        # df.loc[notInSimulatedData, "ExpressionStatus"] = "RealOnly"
        # df.loc[inNeither, "ExpressionStatus"] = "NeithSample"
        # print(df)
        # set up the plotting
        sns.set(style = "whitegrid", palette = "muted", color_codes = True)
        # sns.set(style = "dark", palette = "inferno", color_codes = True)
        # calculate the correlation between the two sets of data.
        # print(df.iloc[ : , 0])
        names = list(df.columns)
        col_one = df.iloc[ : , 1]
        col_two = df.iloc[ : , 2]
        # corr = scipy.stats.pearsonr(col_one, col_two)[0]
        
        sns.scatterplot(x = names[1], y = names[2], hue = 'Lables', marker = '+', style = 'ExpressionStatus', data = df, ax = ax, palette = 'inferno', s = 2.0)
        at = AnchoredText("{inBoth} are found in both real and simulated data\n{neither} expressed in neither real nor simulated data\n{realOnly} are found in real data only\n{simOnly} are found in simulated data only\nCut off threshould: {cut}\nCorr: {corr}".format(
            neither = neither, realOnly = realOnly, simOnly = simOnly, inBoth = inBoth, corr = corr, total = len(df) - realOnly - simOnly + len(inNeither), cut = cutOff), 
                          prop = dict(size = 8), frameon = True, loc = 'upper left')
        at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
   
        ax.add_artist(at)
        ax.set_xlabel("Real Data")
        ax.set_ylabel("Simulated Data")
        ax.legend(loc = 'lower right', fancybox = True, shadow = False, scatterpoints = 200, prop = {'size': 8})


In [75]:
loader = DataLoader()
kall = loader.getKall()

In [76]:
rsem = loader.getRSEM()

In [77]:
test = rsem[0]

In [81]:
kall

,PDX_10Veh,PDX_12Veh,PDX_13P3_40,PDX_14P3_20,PDX_1P3_20,PDX_2P3_40,PDX_3P3_40,PDX_4P3_20,PDX_5P3_40,PDX_6P3_20,PDX_7P3_20,PDX_8Veh,PrimaryTumor
GeneID,,,,,,,,,,,,,
ENSG00000000003,11.986054,10.654686,12.632209,11.984520,12.154459,12.590210,12.347698,12.682657,12.871223,12.345260,12.411518,12.649463,12.064354
ENSG00000000005,6.239048,8.567898,6.607359,6.334044,6.146942,6.083594,5.886587,6.250034,6.234144,6.058683,6.101227,6.355960,6.591030
ENSG00000000419,10.992695,8.771258,11.214416,10.664481,10.790173,11.174510,11.051685,11.486012,11.258944,11.095102,10.942841,11.140087,9.527697
ENSG00000000457,9.080455,10.036320,8.627819,8.391202,8.666030,8.619614,8.353345,8.312060,8.457892,8.509969,8.979662,8.681332,10.249468
ENSG00000000460,9.475529,11.102118,8.738020,8.628993,9.447304,8.912947,8.809385,8.929883,8.723530,9.074169,9.216291,8.634262,10.227561
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000273475,5.551952,6.756068,5.551952,5.551952,5.551952,5.551952,5.551952,5.551952,5.551952,5.814314,5.551952,5.551952,6.375848
ENSG00000273479,5.551952,6.546476,5.551952,5.551952,5.551952,5.551952,5.551952,5.740078,5.551952,5.737973,5.551952,5.551952,5.871356
ENSG00000273482,9.862638,8.866822,9.982666,10.061316,9.863262,9.741165,9.859578,9.734819,9.939446,9.771534,9.946812,9.856527,9.504428


In [89]:
concat = ConcateData(['PDX_12Veh'], test, kall)
concat.setData()

57773 genes have been detected in PDX_12Veh by RSEM
39293 genes have been detected in PDX_12Veh by kallisto
62158 genes in both dataframes
62158 labels have been attached
                GeneID  PDX_12Veh_RSEM  PDX_12Veh_Kallisto  \
0      ENSG00000000003            62.0           10.654686   
1      ENSG00000000005             2.0            8.567898   
2      ENSG00000000419            27.0            8.771258   
3      ENSG00000000457            22.0           10.036320   
4      ENSG00000000460            13.0           11.102118   
...                ...             ...                 ...   
62153  ENSG00000273470             0.0            6.995122   
62154  ENSG00000273475             0.0            6.756068   
62155  ENSG00000273479             0.0            6.546476   
62156  ENSG00000273482             0.0            8.866822   
62157  ENSG00000273490             0.0            5.551952   

                        Label  
0              DetectedByBoth  
1              Detec